<a href="https://colab.research.google.com/github/wizard339/article_finder/blob/main/article_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import urllib.request
import urllib.parse
import argparse
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time

In [3]:
BASE_URL = 'http://export.arxiv.org/api/query?search_query='
PREFIX = {'Title': 'ti',
          'Author': 'au',
          'Abstract': 'abs',
          'Comment': 'co',
          'Journal Reference': 'jr',
          'Subject Category': 'cat',
          'Report Number': 'rn',
          'ID': 'id',
          'All': 'all'}
TIME_PERIOD = {'Day': datetime.timedelta(1),
               'Week': datetime.timedelta(7),
               'Mounth': datetime.timedelta(30),
               'Year': datetime.timedelta(365)}
CURRENT_DATE = datetime.datetime.now()

# input_keywords = input('Please enter the keywords or search phrases separated by commas: ')
input_keywords = 'reinforcement AND learning'

In [14]:
CURRENT_DATE - TIME_PERIOD['Week']

datetime.datetime(2022, 8, 26, 12, 9, 30, 835628)

In [9]:
def make_query(base_url=BASE_URL, prefix=PREFIX['All'], start=0, keywords=input_keywords, time_period=TIME_PERIOD['Week']):
    """нужно ввести счетчик и проверку на значение тега updated в entry"""
    # the starting date from which we receive the data
    starting_date = CURRENT_DATE - time_period
    # encoding of the search query
    keywords = urllib.parse.quote_plus(keywords)

    cols = ['updated', 'title', 'summary', 'author', 'link']
    articles = pd.DataFrame(columns=cols)
    start_counter = 0
    
    while True:
      url = f'{base_url}{prefix}:{keywords}&start={start_counter}&max_results=200&sortBy=lastUpdatedDate&sortOrder=descending'
      print(url)
      req = urllib.request.Request(url)
      with urllib.request.urlopen(req) as response:
          if response.status==200:
              soup = BeautifulSoup(response, 'xml')
          else:
              raise ValueError('Please, check the correctness of the request')
              break
      date_on_the_page = datetime.datetime(*[int(i) for i in soup.entry.updated.string.split('T')[0].split('-')])
      if date_on_the_page >= starting_date:
        print(soup.entry.updated)
        for tag in soup.find_all('entry'):
            updated_to_df = pd.to_datetime(tag.updated.string.split('T')[0])
            authors_list = ", ".join([n.string for n in tag.find_all('name')])
            link_to_df = tag.find(title='pdf').get('href')
            row_to_concat = pd.DataFrame({'updated': updated_to_df,
                                          'title': tag.title,
                                          'summary': tag.summary,
                                          'author': authors_list,
                                          'link': link_to_df}, index=tag.id)
            articles = pd.concat([articles, row_to_concat])
      else:
        break
      start_counter += 1
      time.sleep(3)

    return articles

In [10]:
articles = make_query()

http://export.arxiv.org/api/query?search_query=all:reinforcement+AND+learning&start=0&max_results=200&sortBy=lastUpdatedDate&sortOrder=descending
<updated>2022-09-01T17:35:02Z</updated>
http://export.arxiv.org/api/query?search_query=all:reinforcement+AND+learning&start=1&max_results=200&sortBy=lastUpdatedDate&sortOrder=descending
<updated>2022-09-01T17:03:07Z</updated>
http://export.arxiv.org/api/query?search_query=all:reinforcement+AND+learning&start=2&max_results=200&sortBy=lastUpdatedDate&sortOrder=descending
<updated>2022-09-01T16:49:22Z</updated>
http://export.arxiv.org/api/query?search_query=all:reinforcement+AND+learning&start=3&max_results=200&sortBy=lastUpdatedDate&sortOrder=descending
<updated>2022-09-01T16:31:37Z</updated>
http://export.arxiv.org/api/query?search_query=all:reinforcement+AND+learning&start=4&max_results=200&sortBy=lastUpdatedDate&sortOrder=descending
<updated>2022-09-01T16:28:47Z</updated>
http://export.arxiv.org/api/query?search_query=all:reinforcement+AND+l

In [ ]:
datetime.datetime(*[int(i) for i in soup.entry.updated.string.split('T')[0].split('-')])

datetime.datetime(2022, 8, 31, 0, 0)

In [ ]:
datetime.datetime(2022,8,31)

datetime.datetime(2022, 8, 31, 0, 0)

In [11]:
articles.sample(10)

,updated,title,summary,author,link
http://arxiv.org/abs/2208.08497v1,2022-08-17,Choquet regularization for reinforcement learning,We propose \emph{Choquet regularizers} to me...,"Xia Han, Ruodu Wang, Xun Yu Zhou",http://arxiv.org/pdf/2208.08497v1
http://arxiv.org/abs/2208.11737v1,2022-08-24,Mastering Autonomous Assembly in Fusion Applic...,Robotic peg-in-hole assembly is an essential...,"Ruochen Yin, Huapeng Wu, Ming Li, Yong Cheng, ...",http://arxiv.org/pdf/2208.11737v1
http://arxiv.org/abs/2007.08380v2,2022-08-30,Joint Trajectory and Passive Beamforming Desig...,"In this paper, the intelligent reflecting su...","Liang Wang, Kezhi Wang, Cunhua Pan, Nauman Aslam",http://arxiv.org/pdf/2007.08380v2
http://arxiv.org/abs/2206.12700v2,2022-08-16,Towards Modern Card Games with Large-Scale Act...,Axie infinity is a complicated card game wit...,"Zhiyuan Yao, Tianyu Shi, Site Li, Yiting Xie, ...",http://arxiv.org/pdf/2206.12700v2
http://arxiv.org/abs/2109.09889v2,2022-08-20,A Simple Unified Framework for Anomaly Detecti...,Abnormal states in deep reinforcement learni...,"Hongming Zhang, Ke Sun, Bo Xu, Linglong Kong, ...",http://arxiv.org/pdf/2109.09889v2
http://arxiv.org/abs/2206.14191v2,2022-08-18,Low Emission Building Control with Zero-Shot R...,Heating and cooling systems in buildings acc...,"Scott R. Jeen, Alessandro Abate, Jonathan M. C...",http://arxiv.org/pdf/2206.14191v2
http://arxiv.org/abs/2105.14111v5,2022-08-20,Goal Misgeneralization in Deep Reinforcement L...,"We study goal misgeneralization, a type of o...","Lauro Langosco, Jack Koch, Lee Sharkey, Jacob ...",http://arxiv.org/pdf/2105.14111v5
http://arxiv.org/abs/2208.10483v1,2022-08-22,Prioritizing Samples in Reinforcement Learning...,Most reinforcement learning algorithms take ...,"Shivakanth Sujit, Somjit Nath, Pedro H. M. Bra...",http://arxiv.org/pdf/2208.10483v1
http://arxiv.org/abs/2012.10147v2,2022-08-18,Hierarchical principles of embodied reinforcem...,Cognitive Psychology and related disciplines...,"Manfred Eppe, Christian Gumbsch, Matthias Kerz...",http://arxiv.org/pdf/2012.10147v2
http://arxiv.org/abs/2208.10458v1,2022-08-22,Minimax-Optimal Multi-Agent RL in Zero-Sum Mar...,This paper is concerned with two-player zero...,"Gen Li, Yuejie Chi, Yuting Wei, Yuxin Chen",http://arxiv.org/pdf/2208.10458v1


In [12]:
articles.updated.value_counts()

2022-08-26    1400
2022-08-18    1330
2022-08-22    1190
2022-08-16     984
2022-08-17     910
2022-08-23     840
2022-08-15     808
2022-08-19     770
2022-08-25     770
2022-08-24     700
2022-08-30     675
2022-08-29     630
2022-08-21     560
2022-08-27     405
2022-08-28     369
2022-08-12     354
2022-08-20     350
2022-08-31     270
2022-08-11     231
2022-08-13     150
2022-09-01     136
2022-08-14     123
2022-08-10      45
Name: updated, dtype: int64

In [13]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14000 entries, http://arxiv.org/abs/2201.13170v3 to http://arxiv.org/abs/2208.05120v1
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   updated  14000 non-null  datetime64[ns]
 1   title    14000 non-null  object        
 2   summary  14000 non-null  object        
 3   author   14000 non-null  object        
 4   link     14000 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 656.2+ KB
